# Fire Event Capture and Visualization using Sentinel-2 Imagery

This notebook processes wildfire data using Sentinel-2 imagery. It masks clouds, generates rectangular areas around fire points(~1km), and creates URLs for visualized images.

## Key Functions

- **`earth_engine_init(project_id)`**: Authenticates and initializes the Earth Engine.
- **`mask_s2_clouds(image)`**: Masks clouds in Sentinel-2 images.
- **`load_fire_data(csv_path)`**: Loads fire data from a CSV file.
- **`fetch_image_collection(longitude, latitude, start_date, end_date)`**: Fetches an image collection for a fire event.
- **`generate_image_url(collection, rectangle)`**: Generates a visualization URL for a clipped image.
- **`process_fire_events(fire_df)`**: Processes fire data and retrieves image URLs.
- **`download_images(img_urls)`**: Downloads images from generated URLs.

## Usage

1. Open Script In Colab: https://colab.research.google.com/github/gunelaliyevaa/wildfire-detection-using-satellite-imagery/blob/main/scripts/image_extraction_script.ipynb

2. Set the project ID and CSV paths in the `main()` function.
3. Run `main()` to process the data and download images.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import ee
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed


# Authenticate and initialize Earth Engine
def earth_engine_init(project_id):
    ee.Authenticate()
    ee.Initialize(project = project_id)

# Function to mask clouds using the Sentinel-2 QA60 band
def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask).divide(10000)

# Load the CSV file containing fire data
def load_fire_data(csv_path):
    return pd.read_csv(csv_path)

# Generate a rectangular area around fire points
def create_rectangle(longitude, latitude, buffer=0.02):
    return ee.Geometry.Rectangle([longitude - buffer, latitude - buffer, longitude + buffer, latitude + buffer])

# Get image collection for a given fire event
def fetch_image_collection(longitude, latitude, start_date, end_date):
    rectangle = create_rectangle(longitude, latitude)
    collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                  .filterBounds(rectangle)
                  .filterDate(start_date, end_date)
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
                  .map(mask_s2_clouds))

    return collection, rectangle

# Generate and download an image for a fire event
def download_image_from_event(row, output_dir):
    latitude, longitude = row['latitude'], row['longitude']
    start_date = pd.to_datetime(row['acq_date']).strftime('%Y-%m-%d')
    end_date = (pd.to_datetime(start_date) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')


    collection, rectangle = fetch_image_collection(longitude, latitude, start_date, end_date)
    if collection.size().getInfo() > 0:


        # Select RGB bands and clip the image
        image = collection.median().select(['B4', 'B3', 'B2']).clip(rectangle)

        # Generate download URL
        url = image.getThumbURL({
            'min': 0,
            'max': 0.5,
            'dimensions': 512,
            'region': rectangle,
            'format': 'png'
        })

        # Download and save image
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()
            filename = os.path.join(output_dir, f"{latitude}_{longitude}.png")
            with open(filename, 'wb') as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
            print(f"Downloaded: {filename}")
        except Exception as e:
            print(f"Failed to download image for {latitude}, {longitude}: {e}")

# Process fire events concurrently
def process_fire_events_concurrently(fire_df, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(download_image_from_event, row, output_dir) for _, row in fire_df.iterrows()]
        for future in as_completed(futures):
            future.result()

def main():
    project_id = 'wildfire-detection-ml'
    csv_path = '/content/drive/MyDrive/Copy_2023_fire_archive_SV-C2_539026.csv'
    output_folder = 'SUOMI_VIIRS_IMAGES_2023'

    earth_engine_init(project_id)
    fire_data = load_fire_data(csv_path)

    # Process and download images concurrently
    process_fire_events_concurrently(fire_data, output_folder)


main()


The command `!zip -r my_folder.zip my_folder` creates a compressed ZIP archive of the folder `my_folder`. This will be helpful for downloading the extracted images to your local machine in a single file.

In [ ]:
!zip -r MODIS_Images_2024.zip MODIS_Images_2024

In [ ]:
import shutil

# Specify the folder to remove
folder_to_remove = 'SUOMI_VIIRS_IMAGES_2023'

# Remove the folder and its contents
shutil.rmtree(folder_to_remove)
print(f"Folder '{folder_to_remove}' and its contents have been removed.")

Folder 'SUOMI_VIIRS_IMAGES_2023' and its contents have been removed.
